In [4]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    appName('broadcast-join'). \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/itv007008/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [15]:
spark

In [16]:
### Order Dataset ####
order_base = spark.sparkContext.textFile('/public/trendytech/orders/orders_1gb.csv')

In [17]:
### Customer Dataset ###
customer_base = spark.sparkContext.textFile('/public/trendytech/retail_db/customers/*')

In [18]:
order_mapped = order_base.map(lambda x: (x.split(',')[2],x.split(',')[3]))

In [19]:
customer_mapped = customer_base.map(lambda x: (x.split(',')[0],x.split(',')[8]))

In [20]:
order_mapped.take(1)

[('11599', 'CLOSED')]

In [28]:
customers_broadcast = spark.sparkContext.broadcast(customer_mapped.collect())

In [38]:
def get_pincode(customer_id): 
    try:
        return customer_broadcast.value(customer_id)
    except: 
        return "-1"

In [39]:
join_rdd = order_mapped.map(lambda x: (get_pincode(int (x[0])),x[1]))

In [40]:
join_rdd.take(5)

[('-1', 'CLOSED'),
 ('-1', 'PENDING_PAYMENT'),
 ('-1', 'COMPLETE'),
 ('-1', 'CLOSED'),
 ('-1', 'COMPLETE')]

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 47030)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulators.py", line 268, in handle
    poll(accum_updates)
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/spark-3.0.1-bin-hadoop3.2/python/pyspark/accumulato